In [2]:
import mlflow
import optuna
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from sklearn.pipeline import make_pipeline # Switched to standard sklearn pipeline

# --- 1. Load & Process Data ---
try:
    df = pd.read_csv('reddit_preprocessed.csv')
except FileNotFoundError:
    print("❌ Error: 'reddit_preprocessed.csv' not found. Please upload your dataset.")
    raise

# Basic cleanup
df = df.drop('Unnamed: 0', axis=1, errors='ignore')
df = df.dropna(subset=['clean_comment', 'category']) # Ensure target and text exist
df['clean_comment'] = df['clean_comment'].astype(str)
print(df.shape)

# Encode Labels (-1, 0, 1 -> 0, 1, 2)
le = LabelEncoder()
y = le.fit_transform(df['category'])
class_names = [str(cls) for cls in le.classes_] # Should be ['-1', '0', '1']

# ✅ FIX LEAKAGE: Split Raw Text FIRST
# We split the text strings, NOT the vectors. Vectorization happens inside the pipeline.
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    df['clean_comment'], y, test_size=0.2, random_state=42, stratify=y
)

# --- 2. Optuna Tuning (CPU Mode) ---
mlflow.set_experiment("EXP 8 - LightGBM Internal Class Weight")
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    # Hyperparameters to tune
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'class_weight': 'balanced',  # <--- ACTIVATED: Internal Imbalance Handling
        'objective': 'multiclass',
        'verbose': -1,
        'n_jobs': 1  # 1 job per trial prevents CPU contention
    }

    # Pipeline: Vectorizer -> LGBM (No ADASYN)
    # Increased max_features to 10,000 for better text representation
    clf = make_pipeline(
        TfidfVectorizer(max_features=10000, ngram_range=(1, 2)),
        LGBMClassifier(**params, random_state=42)
    )

    # Cross Val on raw text
    return cross_val_score(clf, X_train_raw, y_train, cv=3, scoring='accuracy').mean()

print("🚀 Tuning LightGBM with Class Weights (CPU Mode)...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

import optuna.visualization as vis

# --- 2.5 Visualization & Logging (Add this block) ---
print("📊 Generating Optuna Visualizations...")

# 1. Optimization History (Did the model improve over time?)
fig_history = vis.plot_optimization_history(study)
fig_history.show() # Display in Colab

# 2. Hyperparameter Importance (Which parameter mattered most?)
fig_importance = vis.plot_param_importances(study)
fig_importance.show() # Display in Colab

# 3. Slice Plot (Visualizing the search space)
fig_slice = vis.plot_slice(study)
fig_slice.show() # Display in Colab

# --- 3. Final Training ---
print("✅ Tuning complete. Training final model...")
best_params = study.best_params

# Ensure class_weight is in the final params (Optuna doesn't suggest it, we forced it)
best_params['class_weight'] = 'balanced'
best_params['objective'] = 'multiclass'

final_pipeline = make_pipeline(
    TfidfVectorizer(max_features=10000, ngram_range=(1, 2)),
    LGBMClassifier(**best_params, random_state=42, verbose=-1, n_jobs=-1) # n_jobs=-1 uses all CPU cores
)

final_pipeline.fit(X_train_raw, y_train)
y_pred = final_pipeline.predict(X_test_raw)

# --- 4. Detailed Logging ---
run_name = "LGBM_Internal_Weight_10k_Features"
with mlflow.start_run(run_name=run_name):
    # Log Parameters
    mlflow.log_params(best_params)
    mlflow.log_param("features", 10000)
    mlflow.log_param("ngram", "1,2")
    mlflow.log_param("imbalance_method", "class_weight='balanced'")

    # Generate Report (output_dict=True for parsing)
    # target_names ensures the report uses "-1", "0", "1" instead of 0, 1, 2
    report_dict = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)

    # Print readable report to console
    print("\n📊 Detailed Classification Report:")
    print(classification_report(y_test, y_pred, target_names=class_names))

    # Log Global Metrics
    mlflow.log_metric("accuracy", report_dict['accuracy'])
    mlflow.log_metric("macro_f1", report_dict['macro avg']['f1-score'])
    mlflow.log_metric("weighted_f1", report_dict['weighted avg']['f1-score'])

    # Log Class-Specific Metrics
    # This loop grabs Precision, Recall, and F1 for EACH class (-1, 0, 1)
    for label in class_names:
        if label in report_dict:
            metrics = report_dict[label]
            safe_label = label.replace("-", "neg_") # MLflow keys play nicer without leading dashes
            mlflow.log_metric(f"class_{safe_label}_precision", metrics['precision'])
            mlflow.log_metric(f"class_{safe_label}_recall", metrics['recall'])
            mlflow.log_metric(f"class_{safe_label}_f1", metrics['f1-score'])

    print(f"✅ MLflow Run Complete. Accuracy: {report_dict['accuracy']:.4f}")

(36662, 2)


2026/02/16 17:40:18 INFO mlflow.tracking.fluent: Experiment with name 'EXP 8 - LightGBM Internal Class Weight' does not exist. Creating a new experiment.


🚀 Tuning LightGBM with Class Weights (CPU Mode)...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut

📊 Generating Optuna Visualizations...


/usr/local/lib/python3.12/dist-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




✅ Tuning complete. Training final model...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names




📊 Detailed Classification Report:
              precision    recall  f1-score   support

          -1       0.81      0.77      0.79      1650
           0       0.85      0.96      0.90      2529
           1       0.91      0.84      0.87      3154

    accuracy                           0.87      7333
   macro avg       0.86      0.86      0.85      7333
weighted avg       0.87      0.87      0.86      7333

✅ MLflow Run Complete. Accuracy: 0.8654
🏃 View run LGBM_Internal_Weight_10k_Features at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/7/runs/1adca0855ebf4e0d8135d2d1c9dd6686
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/7


In [1]:
!pip install mlflow dagshub optuna lightgbm imbalanced-learn kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.2/263.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5

In [1]:
import dagshub
dagshub.init(repo_owner='Ritk-Raikwar', repo_name='reddit-comment-sentiment-analysis', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d9eeaba0-8bbe-45ca-b12e-17af9b248fae&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=7de7b65cdf27869601e7147ade006b28f4cf4b7b9e80c03f0700707b9f76f06d




Output()

Accessing as Ritk-Raikwar

Initialized MLflow to track repo "Ritk-Raikwar/reddit-comment-sentiment-analysis"

Repository Ritk-Raikwar/reddit-comment-sentiment-analysis initialized!